### Find cost of single family homes with given zip codes.

In [1]:
import json
import requests
import pandas as pd
import csv
from config_zillow import api_key
import time

In [2]:
datafile1 = "../data/zipcodes.csv"
df1 = pd.read_csv(datafile1)
len(df1)


407

### Grab zipcode information.

In [52]:
df1.head(10)


,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,29001,STANDARD,0,Alcolu,NaN,NaN,SC,Clarendon County,America/New_York,803,NaN,US,33.76,-80.15,1810
1,29003,STANDARD,0,Bamberg,NaN,Midway,SC,Bamberg County,America/New_York,803,NaN,US,33.29,-81.03,4900
2,29006,STANDARD,0,Batesburg,"Batesburg-leesville, Batsbrg-levil","Holtson Crossroads, Kneece, New Holland Crossr...",SC,Lexington County,America/New_York,803,NaN,US,33.90,-81.54,8070
3,29009,STANDARD,0,Bethune,NaN,NaN,SC,Kershaw County,America/New_York,843,NaN,US,34.41,-80.34,1880
4,29010,STANDARD,0,Bishopville,Wisacky,"Alcot, Ashland, Lucknow, Manville, Mccutchen C...",SC,Lee County,America/New_York,803,NaN,US,34.21,-80.24,8910
5,29014,STANDARD,0,Blackstock,NaN,"Cornwell, Douglass, Stover, Woodward",SC,Chester County,America/New_York,NaN,NaN,US,34.55,-81.15,1480
6,29015,STANDARD,0,Blair,NaN,NaN,SC,Fairfield County,America/New_York,803,NaN,US,34.49,-81.35,1290
7,29016,STANDARD,0,Blythewood,NaN,NaN,SC,Richland County,America/New_York,803,NaN,US,34.21,-80.97,20400
8,29018,STANDARD,0,Bowman,NaN,NaN,SC,Orangeburg County,America/New_York,803,NaN,US,33.34,-80.68,2860
9,29020,STANDARD,0,Camden,NaN,"Antioch, Dusty Bend, Kirkland, Kirkwood, Red H...",SC,Kershaw County,America/New_York,803,NaN,US,34.26,-80.61,19120


### Grab information about single family homes from Zillow.

In [4]:
url = f"http://www.zillow.com/webservice/GetDemographics.htm?zws-id={api_key}&zip="

zip_code = df1["zip"]
new_url = []

for x in zip_code:
    n_url = f"{url}{x}"
    new_url.append(n_url)

text_container = []
for y in new_url:
    time.sleep(0.5)
    try:
        response = requests.get(y)
        response = response.text
        text_container.append(response)
        
    except:
        print ("Not available")


In [91]:
df = pd.DataFrame({
    "Text": text_container
})
df.to_csv("../data/dumpfile.csv")

In [28]:
zip_code = []
bbc = []
for x in text_container:
    try:
        ts = "</city><zip>"
        aa = x.split(ts)
        bb = aa[1].split("<")
        bbc.append(aa[1])
        zip_co = bb[0]
        zip_code.append(zip_co)

    except:
        continue


In [90]:
cost_home = []
for r in bbc:
    s = 'Median Single Family Home Value</name><values><nation><value type="USD">'    
    t = 'Median Single Family Home Value</name><values><zip><value type="USD">'
    try:
        aaa = r.split(t)
        bbb = aaa[1].split("<")
        home_co = bbb[0]
        cost_home.append(home_co)
    except:
        aaaa = r.split(s)
        bbbb = aaaa[1].split("<")
        home_co = bbbb[0]
        
        cost_home.append(home_co)

In [201]:
data = pd.DataFrame({
    "Zip Code": zip_code,
    "Cost of Single Family Home": cost_home
})

data["Zip Code"] = data["Zip Code"].astype("int")
data["Cost of Single Family Home"] = data["Cost of Single Family Home"].astype("float64")

data.to_csv("../data/Single_Family_Home_Cost.csv")
data.head(20)

,Cost of Single Family Home,Zip Code
0,208300.0,29001
1,208300.0,29003
2,80700.0,29009
3,208300.0,29010
4,208300.0,29014
5,87800.0,29015
6,229700.0,29016
7,81000.0,29018
8,110400.0,29020
9,208300.0,29030


### Grab information about single family homes from realtor.com csv file.

In [202]:
datafile2 = "../data/realtor_info.csv"
df2 = pd.read_csv(datafile2)
df2["Median Listing Price"] = df2["Median Listing Price"].astype("float64")
df2["ZipCode"] = df2["ZipCode"].astype("int")


df2 = df2[["ZipCode", "Median Listing Price"]]
df2 = df2.rename(columns ={"ZipCode": "Zip Code"})
df2.head()

,Zip Code,Median Listing Price
0,29003,101800.0
1,29006,116050.0
2,29016,291200.0
3,29020,208550.0
4,29033,138775.0


### Merge the two tables.  Find Average. 

In [203]:
data1 = pd.merge(data, df2, on = "Zip Code", how = "left")
data1.head()

,Cost of Single Family Home,Zip Code,Median Listing Price
0,208300.0,29001,NaN
1,208300.0,29003,101800.0
2,80700.0,29009,NaN
3,208300.0,29010,NaN
4,208300.0,29014,NaN


In [205]:
data1["Median Listing Price"] = data1["Median Listing Price"].fillna(0)
data1.head(300)

,Cost of Single Family Home,Zip Code,Median Listing Price
0,208300.0,29001,0.00
1,208300.0,29003,101800.00
2,80700.0,29009,0.00
3,208300.0,29010,0.00
4,208300.0,29014,0.00
5,87800.0,29015,0.00
6,229700.0,29016,291200.00
7,81000.0,29018,0.00
8,110400.0,29020,208550.00
9,208300.0,29030,0.00


### Conclusion:  Do not use this table (data1) and do not bother with the "df2" table!  The prices are so different and there is a lack of information in that dataset.

# USE "data" table or "Single_Family_Home_Cost.csv"